# TO-DO STEPS PISANE NA PIERWSZYCH ZAJĘCIACH

Data Scraping:


## Data Processing (dla obu zbiorów danych)
1. Ułożenie danych w kolumnach.
2. Miejsca zerowe.
3. Feature Engineering. (kolumna koordynaty geograficzne, 


## Data analysis. (+potencjalnie Streamlit)
1. Prócz metrażu czynniki najbardziej wpływające na cenę.
2. Analiza ceny do piętra
3. Analiza odległości geograficznej od centrum
4. Analiza parking/balon
5. Analiza ceny/trendów w konkretnych dzielniach.


# Jeśli model to.
5. Onehotencoding.
6. Train/Test
7. Model.

Bonus points:
    wizualizacja geograficzna (streamlit - prettymap/mapbox)
    stosunek ilości balkonów w mieszkaniu na dzielnice (balkon rate)
    wpływ i częstotliwość '!' w nazwie oferty i ich zestawienie z ceną
    

# TO-DO 16.04.2024

1. Docelowo proponuję, żeby pierwszy etap (pisany w jednym notatniku jupiter) zamknął się po data preparation i data engeenering, po tym etapie zapiszemy dane do plików do dalszych analiz (wielkrotne pobieranie koordynatów może być bolesne). Aby to osiągnąć trzeba:
  - Uspójnić notatniki (na razie jeszcze nie dane :P), tak, byśmy pracowali na jednym.
  - Dodać na początek chaina uzupełnienie zbiorów o nowe dane (można wyciąć fragmenty z utils) - tam, gdzie będziemy analizować dane statycznie i/lub przestrzennie (pierwotny plan).
  - Jeśli idzie o analize w szeregach czasowych, to trochę inne para kaloszy, można zostawić na później -- chyba, że ktorś będzie chciał się tym zająć.
  - Uważam, że trzeba wybrać, które analizy przeprowadzamy na uspójnionych z obu zbiorów danych, a które osobno na zbiorach - otodom ma więcej cech, nieruchomości online (chyba) bardziej rozróżnia pewne cechy (np. te garaże) - Do decyzji.
  - Z obu zbiorów usunąć "duplikaty" - różne ogłoszenia tego samego mieszkania. Tu trzeba będzie przyjąć jakąś heurystykę.
  - Uzupełnić koordynaty
  - Na ich podstawie dodać kolumnę z odległością od wyróżnionego miejsca (centrum) w metrach.
  - Zbiory zapisać


  - Jeśli idzie o szeregi czasowe, to IMO data i cena w badanych punktach czasowych starczy (cokolwiek innego się tam jeszcze mogło zmieniać). Będzie trzeba wybrać tylko te oferty, które zmieniały się w tej kolumnie.

2. Etap analiz:
- Sprawdzić, co z labów Czecha jest do wykorzystania, bo mam przeczycie, że coś z tego, co u nie go robiliśmy się przyda - PCA chociażby i ten (dual) scatterplot.
- Z drugich labów Czecha ten pearson corelation, to macierz korelacji, którą i tak planowaliśmy.
- Na SISI będziemy trenować model drzew decyzyjnych, więc można go tu dodać też.
- Trzeba przejrzeć, co jeszcze planowaliśmy i co jest w "literaturze" (bo znalzałem coś takiego), co jeszcze można przeprowadzić.


3. Wizualizacja:
Poza oczywstymi wykresikami, tabelami itp, ta heatmapa (oraz heatmapa+, jak ze stramlita) wydaje mi się fajnym bajerem. Można spróbować się podjąć. W oógle wydaje mi się, że w tym aspekcie geograficznym można będzie fajnie nie tylko przeprowadzać analizy, ale też pokazywać rzeczy.

Ważna rzecz to dokumentowanie prac (albo w readme, albo zacząć odrębny dokument) nawet, gdy nic nie wychodzi.


To są oczywiście moje wstępne i ogólne pomysły i jeśli macie inne, to śmiało wędrujcie w te strony, które Wam się wydadzą sensowne i słuszne.

    

# Zbiór Otodom

## Załadowanie i agregacja danych

In [80]:
import numpy as np
import pandas as pd

In [81]:
df_otodom = pd.read_csv('datasets/otodom_dataset_raw.csv')
df_2024_03_25_otodom_new_data = pd.read_csv('datasets/timestamps/2024-03-25-otodom-new_data.csv')
df_2024_03_25_otodom_updated_data = pd.read_csv('datasets/timestamps/2024-03-25-otodom-updated_data.csv')
df_2024_04_07_otodom_new_data = pd.read_csv('datasets/timestamps/2024-04-07-otodom-new_data.csv')
df_2024_04_07_otodom_updated_data = pd.read_csv('datasets/timestamps/2024-04-07-otodom-updated_data.csv')
df_2024_04_21_otodom_new_data = pd.read_csv('datasets/timestamps/2024-04-21-otodom-new_data.csv')
df_2024_04_21_otodom_updated_data = pd.read_csv('datasets/timestamps/2024-04-21-otodom-updated_data.csv')

In [82]:
def update_dataframe(oldest_df, updated_df, new_df):
    def set_url_as_index(df):
        if 'url' in df.columns:
            df.set_index('url', inplace=True)
        else:
            raise ValueError("URL column not found. Please check the column names.")
    try:
        set_url_as_index(oldest_df)
        set_url_as_index(updated_df)
        set_url_as_index(new_df)
    except ValueError as e:
        print(e)
        return None
    
    oldest_df.update(updated_df)
    merged_df = pd.concat([oldest_df, new_df], axis=0)
    merged_df.reset_index(inplace=True)

    return merged_df

In [83]:
df_otodom = update_dataframe(df_otodom, df_2024_03_25_otodom_updated_data, df_2024_03_25_otodom_new_data)
df_otodom = update_dataframe(df_otodom, df_2024_04_07_otodom_updated_data, df_2024_04_07_otodom_new_data)
df_otodom = update_dataframe(df_otodom, df_2024_04_21_otodom_updated_data, df_2024_04_21_otodom_new_data)

In [84]:
df_otodom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12672 entries, 0 to 12671
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   url                   12672 non-null  object
 1   name/title            12672 non-null  object
 2   address               12672 non-null  object
 3   price                 12672 non-null  object
 4   area                  12672 non-null  object
 5   price-per-area        12092 non-null  object
 6   floor/store           12433 non-null  object
 7   no of rooms           12671 non-null  object
 8   year of construction  11304 non-null  object
 9   parking space         7625 non-null   object
 10  market                11304 non-null  object
 11  form of ownership     9617 non-null   object
 12  condition             10077 non-null  object
 13  rent                  5844 non-null   object
 14  heating               9244 non-null   object
 15  advertiser type       11304 non-null

## Przygotowanie danych

### Cena

#### Usunięcie brakujących danych, uspójnienie oraz zmiana na typy liczbowe

In [85]:
# Wierszy bez ceny
no_price = df_otodom[~df_otodom['price'].str.contains('\d')]
no_price['price'].count()

583

In [86]:
# Usunięcie wierszy z brakującą ceną (no_price)
df_otodom.drop(no_price.index, inplace=True)
df_otodom['price'].count()

12089

#### Ceny podane w innej walucie

In [87]:
# Wiersze w EUR
different_currency = df_otodom[~df_otodom['price'].str.contains('zł', na=False)].copy()
different_currency['price'].count()

8

In [88]:
different_currency['price-per-area'] = different_currency['price-per-area'].str.replace(' ', '').str.extract(r'(\d+\.\d+|\d+)').astype(float)
different_currency['area'] = different_currency['area'].str.replace(',', '.').str.extract(r'(\d+\.\d+|\d+)').astype(float)

In [89]:
df_otodom.loc[different_currency.index]

,url,name/title,address,price,area,price-per-area,floor/store,no of rooms,year of construction,parking space,market,form of ownership,condition,rent,heating,advertiser type,elevator,outdoor area,building type,building material
1265,https://www.otodom.pl/pl/oferta/penthouse-232m...,Penthouse 232m w zabytkowej kamienicy Stare Mi...,"Nowy Świat, Stare Miasto, Kraków, małopolskie",1 149 999 EUR,"232,1 m²",21 368 zł/m²,4/4,5,1906,NaN,wtórny,pełna własność,do zamieszkania,NaN,gazowe,biuro nieruchomości,nie,NaN,kamienica,cegła
2056,https://www.otodom.pl/pl/oferta/penthouse-w-po...,Penthouse w pobliżu Rynku Głównego,"marszałka JózefaPiłsudskiego, Nowy Świat, Star...",1 150 000 EUR,232 m²,21 377 zł/m²,4/5,9,1906,NaN,wtórny,pełna własność,do zamieszkania,1 700 zł,gazowe,biuro nieruchomości,tak,"balkon, taras",kamienica,brak informacji
3144,https://www.otodom.pl/pl/oferta/5-pokoi-taras-...,5 pokoi | Taras 77m2 | Komórka | Centrum | Mar...,"ul. marsz. Józefa Piłsudskiego, Nowy Świat, St...",1 149 999 EUR,"232,1 m²",21 368 zł/m²,4/4,5,1908,garaż/miejsce parkingowe,wtórny,pełna własność,do zamieszkania,850 zł,gazowe,biuro nieruchomości,tak,"taras, balkon",kamienica,cegła
3977,https://www.otodom.pl/pl/oferta/chorwacja-spli...,Chorwacja Split luksusowy apartament,"Krowodrza, Krowodrza, Kraków, małopolskie",585 280 EUR,"91,45 m²",27 601 zł/m²,3/4,4,2025,garaż/miejsce parkingowe,pierwotny,pełna własność,do zamieszkania,100 EUR,elektryczne,prywatny,tak,"balkon, taras",apartamentowiec,silikat
4722,https://www.otodom.pl/pl/oferta/penthouse-tara...,Penthouse tarasem na dachu w kamienicy z windą,"Marszałka Józefa Piłsudskiego, Nowy Świat, Sta...",1 150 000 EUR,"228,7 m²",21 686 zł/m²,4/5,5,1906,NaN,wtórny,pełna własność,do zamieszkania,NaN,NaN,biuro nieruchomości,tak,"balkon, taras",kamienica,brak informacji
4908,https://www.otodom.pl/pl/oferta/penthouse-w-od...,Penthouse w odrestaurowanej kamienicy - taras ...,"ul. marsz. Józefa Piłsudskiego, Piasek, Stare ...",1 150 000 EUR,228 m²,21 896 zł/m²,4/5,5.0,1906,garaż/miejsce parkingowe,wtórny,pełna własność,do zamieszkania,220 zł,gazowe,biuro nieruchomości,tak,"taras, balkon",kamienica,cegła
4995,https://www.otodom.pl/pl/oferta/penthouse-star...,Penthouse Stare Miasto II Poziomy. Taras 70m2,"Marszałka Józefa Piłsudskiego, Nowy Świat, Sta...",1 150 000 EUR,"232,1 m²",21 368 zł/m²,4/5,5,1906,NaN,wtórny,NaN,NaN,NaN,NaN,biuro nieruchomości,nie,NaN,brak informacji,brak informacji
9359,https://www.otodom.pl/pl/oferta/penthouse-w-od...,Penthouse w odrestaurowanej kamienicy - taras ...,"ul. marsz. Józefa Piłsudskiego, Piasek, Stare ...",1 150 000 EUR,228 m²,21 752 zł/m²,4/5,5,1906,garaż/miejsce parkingowe,wtórny,pełna własność,do zamieszkania,220 zł,gazowe,biuro nieruchomości,tak,"taras, balkon",kamienica,cegła


In [90]:
# zamiana cen na wartości liczbowe
df_otodom['price'] = df_otodom['price'].str.replace(' ', '').str.replace(',', '.')
df_otodom['price'] = df_otodom['price'].str.extract('(\d+\.\d+|\d+)').astype(float)

In [91]:
# przeliczanie ceny na podstawie metrażu i ceny za metr podanej w PLN
different_currency.loc[:, 'price'] = (different_currency['price-per-area'] * different_currency['area'])
df_otodom.loc[different_currency.index, 'price'] = different_currency['price'].astype(float)
df_otodom.loc[different_currency.index, 'price']

1265    4959512.80
2056    4959464.00
3144    4959512.80
3977    2524111.45
4722    4959588.20
4908    4992288.00
4995    4959512.80
9359    4959456.00
Name: price, dtype: float64

In [92]:
# Usunięcie złapanego wiersza z mieszkaniem z Chorwacji
df_otodom.drop(index=3977, inplace=True)

In [93]:
df_otodom['price'].count()

12088

### Zamiana wartości w kolumnach na typy liczbowe

#### metraż

In [94]:
df_otodom['area'] = df_otodom['area'].str.replace(' m²', '')
df_otodom['area'] = df_otodom['area'].str.replace(' ', '')
df_otodom['area'] = df_otodom['area'].str.replace(',', '.').astype(float)

#### cena za metr

In [95]:
# df_otodom[df_otodom['price-per-area'].str.contains(r'\d', na=False).shape[0] & ~df_otodom['price-per-area'].str.contains('zł', na=False)].shape[0]
df_otodom[(df_otodom['price-per-area'].str.contains(r'\d', na=False)) & (~df_otodom['price-per-area'].str.contains('zł', na=False))].shape[0]

0

In [96]:
df_otodom[df_otodom['price-per-area'].isnull()].shape[0]

0

In [97]:
df_otodom['price-per-area'] = df_otodom['price-per-area'].str.replace(' zł/m²', '').str.replace(' ', '').str.replace(',', '.')
df_otodom['price-per-area'] = df_otodom['price-per-area'].astype(float)

#### czynsz

In [98]:
# Wiersze w innej walucie
df_otodom[df_otodom['rent'].str.contains(r'\d', na=False) & ~df_otodom['rent'].str.contains('zł', na=False)]

,url,name/title,address,price,area,price-per-area,floor/store,no of rooms,year of construction,parking space,market,form of ownership,condition,rent,heating,advertiser type,elevator,outdoor area,building type,building material
6335,https://www.otodom.pl/pl/oferta/2m-wysoki-stan...,2M! Wysoki Standard| Gotowe| Balkon | Widok!,"Wola Duchacka, Podgórze Duchackie, Kraków, mał...",620000.0,35.55,17440.0,7/11,2,2019,NaN,wtórny,pełna własność,do zamieszkania,420 GBP,miejskie,biuro nieruchomości,tak,balkon,blok,brak informacji


In [99]:
eur_rent_price = df_otodom[df_otodom['rent'].str.contains('EUR', na=False)]
gbp_rent_price = df_otodom[df_otodom['rent'].str.contains('GBP', na=False)]

In [100]:
# zamiana wartości rent na typ liczbowy
df_otodom['rent'] = df_otodom['rent'].str.replace(' ', '').str.replace(',', '.').str.extract('(\d+\.\d+|\d+)').astype(float)

In [101]:
import yfinance as yf

eur = yf.Ticker("EURPLN=X")
gbp = yf.Ticker("GBPPLN=X")
euro_exchange_rate = eur.info['fiftyDayAverage']
gbp_exchange_rate = gbp.info['fiftyDayAverage']
print(euro_exchange_rate)
print(gbp_exchange_rate)

4.305423
5.0342665


In [102]:
# euro_exchange_rate = 4.28
# gbp_exchange_rate = 4.99

df_otodom.loc[eur_rent_price.index, 'rent'] *= euro_exchange_rate
df_otodom.loc[gbp_rent_price.index, 'rent'] *= gbp_exchange_rate

In [103]:
# Średnia po wycięciu utliersów do ew. uzupełnienia
mean_rent = df_otodom['rent'].mean()
std_rent = df_otodom['rent'].std()

filtered_std_rent = df_otodom[(df_otodom['rent'] > (mean_rent - std_rent)) & 
                        (df_otodom['rent'] < (mean_rent + std_rent))]

new_mean_rent = filtered_std_rent['rent'].mean()
new_mean_rent

541.131739758603

#### liczba pokoi

In [104]:
df_otodom['no of rooms'] = df_otodom['no of rooms'].str.replace(' ', '')
print(df_otodom[df_otodom['no of rooms'].isna()].shape[0])
df_otodom.dropna(subset=['no of rooms'], inplace=True)

346


In [105]:
df_otodom['no of rooms'] = df_otodom['no of rooms'].str.extract('(\d+)').astype(int)

#### piętro

In [106]:
# Rozdzielenie kolumny 'floor/store' na dwie osobne kolumny
df_otodom[['floor/store', 'no of floors/stores in the building']] = df_otodom['floor/store'].str.split('/', expand=True)

In [107]:
# Usunięcie wierszy bez informacji o piętrze
print(df_otodom[df_otodom['floor/store'].isnull()].shape[0])
df_otodom = df_otodom.dropna(subset=['floor/store'])

236


In [108]:
# Zamiana wartości "parter" na 0 w kolumnie "floor"
df_otodom['floor/store'] = df_otodom['floor/store'].replace('parter', 0)

In [109]:
# Zamiana wartości "suterena" na -1 w kolumnie "floor"
df_otodom['floor/store'] = df_otodom['floor/store'].replace('suterena', -1)

In [110]:
# Konwersja kolumny "floor" na typ liczbowy, zamieniając nieliczbowe wartości na NaN
df_otodom['floor/store'] = pd.to_numeric(df_otodom['floor/store'], errors='coerce')

# Usunięcie wierszy zawierających wartości NaN w kolumnie "floor"
df_otodom = df_otodom[~df_otodom['floor/store'].isna()]

In [111]:
df_otodom['floor/store'] = df_otodom['floor/store'].astype(int)

#### liczba pięter

In [112]:
print(df_otodom[df_otodom['no of floors/stores in the building'].isnull()].shape[0])
df_otodom = df_otodom.dropna(subset=['no of floors/stores in the building'])

448


In [113]:
df_otodom['no of floors/stores in the building'] = df_otodom['no of floors/stores in the building'].astype(int)

In [114]:
#Reorder
columns = list(df_otodom.columns)
columns.remove('no of floors/stores in the building')
target_index = columns.index('floor/store')
columns.insert(target_index + 1, 'no of floors/stores in the building')
df_otodom = df_otodom[columns]

In [115]:
# Według wikipedii najwyższy mieszkalny budunek w Krk ma 19 pięter
# https://pl.wikipedia.org/wiki/Lista_najwyższych_budynków_w_Krakowie
print(df_otodom[df_otodom['no of floors/stores in the building'] > 19].shape[0])
df_otodom.drop(df_otodom[df_otodom['no of floors/stores in the building'] > 19].index, inplace=True)

1


#### rok budowy

In [116]:
df_otodom['year of construction'] = pd.to_numeric(df_otodom['year of construction'], errors='coerce')

missing_below_1200 = df_otodom[df_otodom['year of construction'] < 1250]
missing_above_2026 = df_otodom[df_otodom['year of construction'] > 2026]
pd.concat([missing_above_2026, missing_below_1200])

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,...,market,form of ownership,condition,rent,heating,advertiser type,elevator,outdoor area,building type,building material
187,https://www.otodom.pl/pl/oferta/4-pokoje-nowe-...,"4 pokoje, nowe, Promocja -175 tys. taniej","Stare Podgórze, Podgórze, Kraków, małopolskie",998358.0,62.10,16077.0,1,4,4,2924.0,...,pierwotny,pełna własność,do wykończenia,NaN,miejskie,biuro nieruchomości,tak,balkon,apartamentowiec,brak informacji
7888,https://www.otodom.pl/pl/oferta/mieszkanie-na-...,"Mieszkanie na sprzedaż, Kraków 40,2m2","ul. Janusza Supniewskiego, Osiedle Oficerskie,...",699000.0,40.20,17388.0,0,2,2,20100.0,...,wtórny,pełna własność,do zamieszkania,350.0,miejskie,prywatny,nie,NaN,blok,cegła
10812,https://www.otodom.pl/pl/oferta/widokowe-3-pok...,"Widokowe 3 pok, 2 loggie, garaż podziemny, Ban...","ul. Stefana Banacha, Górka Narodowa, Prądnik B...",759000.0,49.47,15343.0,6,8,3,2924.0,...,pierwotny,pełna własność,do wykończenia,1.0,miejskie,biuro nieruchomości,tak,NaN,blok,brak informacji
6685,https://www.otodom.pl/pl/oferta/nowe-3-pokoje-...,Nowe: 3 pokoje na Ruczaju. Garaż podziemny.,"Ruczaj, Dębniki, Kraków, małopolskie",790300.0,56.45,14000.0,3,3,3,1024.0,...,pierwotny,pełna własność,do wykończenia,250.0,miejskie,biuro nieruchomości,tak,balkon,blok,pustak
11292,https://www.otodom.pl/pl/oferta/3-pokoje-na-bi...,3 Pokoje na Bieńczycach! Wysoki Standard!,"os. Wysokie, Bieńczyce, Bieńczyce, Kraków, mał...",669000.0,45.00,14867.0,3,4,3,450.0,...,wtórny,pełna własność,do zamieszkania,450.0,miejskie,biuro nieruchomości,nie,NaN,blok,cegła
11328,https://www.otodom.pl/pl/oferta/mieszkanie-na-...,Mieszkanie na sprzedaż- Mogilska-76 m2 + garaż,"ul. Mogilska, Osiedle Oficerskie, Grzegórzki, ...",1080000.0,76.00,14211.0,8,10,3,87.0,...,wtórny,pełna własność,do zamieszkania,1200.0,miejskie,biuro nieruchomości,tak,balkon,blok,wielka płyta


In [117]:
# Poprawianie "z placa"
df_otodom['year of construction'] = df_otodom['year of construction'].replace(2924.0, 2024)
df_otodom['year of construction'] = df_otodom['year of construction'].replace(1024.0, 2024)
df_otodom['year of construction'] = df_otodom['year of construction'].replace(20100.0, 2010)
df_otodom['year of construction'] = df_otodom['year of construction'].replace(87.0, 1987)
df_otodom.drop(df_otodom[df_otodom['year of construction'] == 450.0].index, inplace=True)

In [118]:
# df_otodom['year of construction'] = df_otodom['year of construction'].fillna(-1).astype(int)
df_otodom['year of construction'].value_counts(dropna=False).head()

year of construction
NaN       2159
2024.0    1740
2023.0    1486
2025.0     483
2022.0     357
Name: count, dtype: int64

In [119]:
# Średnia po wycięciu utliersów do ew. uzupełnienia
mean_year = df_otodom['year of construction'].mean()
std_year = df_otodom['year of construction'].std()

filtered_df = df_otodom[(df_otodom['year of construction'] > (mean_year - std_year)) & 
                        (df_otodom['year of construction'] < (mean_year + std_year))]

new_mean_year = filtered_df['year of construction'].mean()
new_mean_year = int(new_mean_year)
new_mean_year

2013

#### rynek

In [120]:
df_otodom.loc[(df_otodom['market'].isna()) & (df_otodom['year of construction'] > 2022), 'market'] = 'pierwotny'
df_otodom.loc[(df_otodom['market'].isna()) & (df_otodom['year of construction'] < 2023), 'market'] = 'wtórny'

In [121]:
print(df_otodom['market'].isna().sum())
df_otodom.dropna(subset=['market'], inplace=True)

845


In [122]:
df_otodom['market'].value_counts(dropna=False)

market
wtórny       6528
pierwotny    3587
Name: count, dtype: int64

#### rodzaj własności

In [123]:
df_otodom['form of ownership'].value_counts(dropna=False)

form of ownership
pełna własność                       7007
NaN                                  2751
spółdzielcze wł. prawo do lokalu      252
udział                                101
użytkowanie wieczyste / dzierżawa       4
Name: count, dtype: int64

In [124]:
# Zgodnie z obowiązującym prawem 2007 r. nie ma możliwości ustanawiania spółdzielczego własnościowego prawa do lokalu
df_otodom.loc[(df_otodom['form of ownership'].isna()) & (df_otodom['year of construction'] > 2006), 'form of ownership'] = 'pełna własność'

#### stan

In [125]:
df_otodom['condition'].value_counts(dropna=False)

condition
do zamieszkania    4247
do wykończenia     3115
NaN                2311
do remontu          442
Name: count, dtype: int64

#### parking

In [126]:
df_otodom['parking space'].value_counts(dropna=False)

parking space
garaż/miejsce parkingowe    5641
NaN                         4474
Name: count, dtype: int64

In [127]:
df_otodom['parking space'] = df_otodom['parking space'].notna()
df_otodom['parking space'].value_counts(dropna=False)

parking space
True     5641
False    4474
Name: count, dtype: int64

#### Ogrzewanie

In [128]:
df_otodom['heating'].value_counts(dropna=False)

heating
miejskie         6378
NaN              1935
gazowe           1139
inne              348
elektryczne       183
kotłownia         130
piece kaflowe       2
Name: count, dtype: int64

#### balkon

In [129]:
df_otodom['outdoor area'].value_counts(dropna=False)

outdoor area
balkon                    5578
NaN                       2501
ogródek                    508
balkon, taras              455
balkon, ogródek, taras     340
taras                      260
ogródek, taras             250
balkon, ogródek            173
taras, balkon               31
taras, ogródek              16
taras, balkon, ogródek       3
Name: count, dtype: int64

In [130]:
df_otodom['outdoor area'] = df_otodom['outdoor area'].fillna('None')

def standardize_outdoor_area(area):
    if area == 'None':
        return area
    parts = sorted(set(area.replace(' ', '').split(',')))
    return ', '.join(parts)

df_otodom['outdoor area'] = df_otodom['outdoor area'].apply(standardize_outdoor_area)
# outdoor_encoded = df_otodom['outdoor area'].str.get_dummies(sep=', ')
# df_otodom = df_otodom.join(outdoor_encoded)

#### ogłoszeniodawca

In [131]:
df_otodom['advertiser type'].value_counts(dropna=False)

advertiser type
biuro nieruchomości    8864
prywatny               1100
deweloper               151
Name: count, dtype: int64

In [132]:
df_otodom.loc[(df_otodom['advertiser type'].isna()), 'advertiser type'] = 'biuro nieruchomości'

#### winda

In [133]:
df_otodom['elevator'].value_counts(dropna=False)

elevator
tak    5462
nie    4653
Name: count, dtype: int64

In [134]:
# Zgodnie z obowiązującym od 1960 prawem budowalnym
df_otodom.loc[df_otodom['elevator'].isna() & (df_otodom['no of floors/stores in the building'] > 3), 'elevator'] = 'tak'
df_otodom['elevator'] = df_otodom['elevator'].map({'tak': True, 'nie': False, np.nan: False})
df_otodom['elevator'] = df_otodom['elevator'].astype(bool)

#### typ budynku

In [135]:
df_otodom['building type'].value_counts(dropna=False)

building type
blok                5027
apartamentowiec     2031
brak informacji     1754
kamienica           1016
dom wolnostojący     191
szeregowiec           86
plomba                 8
loft                   2
Name: count, dtype: int64

#### materiał

In [136]:
df_otodom['building type'].value_counts(dropna=False)

building type
blok                5027
apartamentowiec     2031
brak informacji     1754
kamienica           1016
dom wolnostojący     191
szeregowiec           86
plomba                 8
loft                   2
Name: count, dtype: int64

In [137]:
df_otodom.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10115 entries, 0 to 12671
Data columns (total 21 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   url                                  10115 non-null  object 
 1   name/title                           10115 non-null  object 
 2   address                              10115 non-null  object 
 3   price                                10115 non-null  float64
 4   area                                 10115 non-null  float64
 5   price-per-area                       10115 non-null  float64
 6   floor/store                          10115 non-null  int32  
 7   no of floors/stores in the building  10115 non-null  int32  
 8   no of rooms                          10115 non-null  int32  
 9   year of construction                 8801 non-null   float64
 10  parking space                        10115 non-null  bool   
 11  market                           

In [138]:
nan_row_count = df_otodom.isna().any(axis=1).sum()
df_otodom.shape[0] - nan_row_count

3652

## Duplikaty

In [139]:
def check_similarity(group):
    if len(group) > 1:
        price_mean = group['price'].mean()
        area_mean = group['area'].mean()
        price_range = price_mean * 0.01
        area_range = area_mean * 0.01
        similar_price = group['price'].between(group['price'] - price_range, group['price'] + price_range)
        similar_area = group['area'].between(group['area'] - area_range, group['area']+ area_range)
        group['is_duplicate'] = similar_price & similar_area
    else:
        group['is_duplicate'] = False
    return group

df_otodom = df_otodom.groupby(['address', 'floor/store', 'no of floors/stores in the building', 'no of rooms']).apply(check_similarity).reset_index(drop=True)

duplicates = df_otodom[df_otodom['is_duplicate']]
non_duplicates = df_otodom[~df_otodom['is_duplicate']]

most_complete_duplicates = duplicates.groupby(['address', 'floor/store', 'no of floors/stores in the building', 'no of rooms']).apply(lambda x: x.loc[x.notna().sum(axis=1).idxmax()]).reset_index(drop=True)

filtered_df_otodom = pd.concat([non_duplicates, most_complete_duplicates], ignore_index=True)

sorted_filtered_df_otodom = filtered_df_otodom.sort_values(by=['address', 'price', 'area'])
sorted_filtered_df_otodom


C:\Users\Piotr\AppData\Local\Temp\ipykernel_24120\1680486895.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_otodom = df_otodom.groupby(['address', 'floor/store', 'no of floors/stores in the building', 'no of rooms']).apply(check_similarity).reset_index(drop=True)
C:\Users\Piotr\AppData\Local\Temp\ipykernel_24120\1680486895.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  most_complete_duplicate

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,...,form of ownership,condition,rent,heating,advertiser type,elevator,outdoor area,building type,building material,is_duplicate
0,https://www.otodom.pl/pl/oferta/nowe-mieszkani...,Nowe mieszkania na Górce Narodowej,"29 listopada - okolice, Górka Narodowa, Prądni...",778689.0,61.30,12703.0,1,5,2,2023.0,...,pełna własność,NaN,NaN,miejskie,biuro nieruchomości,True,balkon,brak informacji,brak informacji,False
1,https://www.otodom.pl/pl/oferta/balkon-3-pokoj...,balkon + 3 pokoje + parking,"Adama Vetulaniego, Prądnik Biały, Prądnik Biał...",970000.0,55.00,17636.0,4,7,3,2023.0,...,pełna własność,NaN,450.0,miejskie,biuro nieruchomości,True,balkon,blok,brak informacji,False
3788,https://www.otodom.pl/pl/oferta/zlocien-2-poko...,"Złocień, 2 pokoje, balkon","Agatowa, Domagały,, Złocień, Bieżanów-Prokocim...",625000.0,36.80,16984.0,4,4,2,2019.0,...,pełna własność,do zamieszkania,425.0,miejskie,biuro nieruchomości,True,balkon,blok,brak informacji,True
2,https://www.otodom.pl/pl/oferta/komfortowe-2-p...,komfortowe 2-pokojowe mieszkanie 42m2,"Albertyńskie, Bieńczyce, Bieńczyce, Kraków, ma...",559000.0,42.00,13310.0,2,4,2,1964.0,...,NaN,NaN,500.0,miejskie,biuro nieruchomości,False,balkon,blok,brak informacji,False
3,https://www.otodom.pl/pl/oferta/aleja-29-listo...,Aleja 29 Listopada obok UR,"Aleja 29 Listopada, Górka Narodowa, Prądnik Bi...",695000.0,48.10,14449.0,3,3,2,NaN,...,pełna własność,do zamieszkania,NaN,gazowe,biuro nieruchomości,False,balkon,blok,brak informacji,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3779,https://www.otodom.pl/pl/oferta/2-pokojowe-mie...,"2-pokojowe mieszkanie 36,75 m2 ul.Siemaszki","Żabiniec, Prądnik Biały, Kraków, małopolskie",735000.0,36.75,20000.0,0,4,2,1970.0,...,pełna własność,do zamieszkania,530.0,miejskie,prywatny,False,balkon,blok,brak informacji,False
3785,https://www.otodom.pl/pl/oferta/3-pokoje-na-za...,3 pokoje na Żabińcu !!!,"Żabiniec, Prądnik Biały, Kraków, małopolskie",850000.0,77.00,11039.0,4,6,3,2002.0,...,pełna własność,NaN,800.0,miejskie,biuro nieruchomości,True,balkon,blok,brak informacji,False
5716,https://www.otodom.pl/pl/oferta/mieszkanie-73-...,"Mieszkanie, 73 m², Kraków","Żabiniec, Prądnik Biały, Kraków, małopolskie",1100000.0,73.00,15068.0,0,6,3,2005.0,...,pełna własność,NaN,700.0,miejskie,biuro nieruchomości,True,balkon,blok,brak informacji,True
3786,https://www.otodom.pl/pl/oferta/widokowe-m3-z-...,"Widokowe M3 z klimatyzacją i parkingiem, Żabiniec","Żabiniec, Prądnik Biały, Kraków, małopolskie",1179000.0,71.50,16490.0,5,8,3,2001.0,...,pełna własność,do zamieszkania,940.0,miejskie,biuro nieruchomości,True,balkon,apartamentowiec,brak informacji,False


In [140]:
sorted_filtered_df_otodom.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5717 entries, 0 to 3787
Data columns (total 22 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   url                                  5717 non-null   object 
 1   name/title                           5717 non-null   object 
 2   address                              5717 non-null   object 
 3   price                                5717 non-null   float64
 4   area                                 5717 non-null   float64
 5   price-per-area                       5717 non-null   float64
 6   floor/store                          5717 non-null   int32  
 7   no of floors/stores in the building  5717 non-null   int32  
 8   no of rooms                          5717 non-null   int32  
 9   year of construction                 5023 non-null   float64
 10  parking space                        5717 non-null   bool   
 11  market                             

In [186]:
nan_row_count = sorted_filtered_df_otodom.isna().any(axis=1).sum()
sorted_filtered_df_otodom.shape[0] - nan_row_count

2540

In [142]:
sorted_filtered_df_otodom.describe()

,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,rent
count,5.717000e+03,5717.000000,5717.000000,5717.000000,5717.000000,5717.000000,5023.000000,3526.000000
mean,9.834145e+05,229.623885,16694.117544,2.387266,4.804968,2.641945,1998.236910,613.069740
std,7.136374e+05,12894.236570,4733.735962,2.129904,2.508053,1.116865,38.831099,3369.267116
min,2.000000e+03,11.700000,1.000000,-1.000000,1.000000,1.000000,1310.000000,0.010000
25%,6.490000e+05,40.000000,13849.000000,1.000000,3.000000,2.000000,1980.000000,400.000000
50%,8.040000e+05,52.000000,15920.000000,2.000000,4.000000,3.000000,2016.000000,530.000000
75%,1.092000e+06,68.000000,18506.000000,3.000000,6.000000,3.000000,2023.000000,700.000000
max,1.350000e+07,975000.000000,70000.000000,10.000000,16.000000,10.000000,2026.000000,200000.000000


## Adresy

In [143]:
import re

# Regex pattern adjusted to exclude capturing prefixes directly
pattern = r'(ul\.|Aleja|aleja|pl\.|al\.)\s*([^,\d]+[\d]*\b)'

def update_address(row):
    if pd.isna(row['name/title']):
        return row['address']

    match = re.search(pattern, row['name/title'])
    if match:
        street_name = match.group(2).strip()
        if pd.isna(row['address']):
            updated_address = street_name
        else:
            if street_name.lower() not in row['address'].lower():
                updated_address = f"{street_name}, {row['address']}"
            else:
                updated_address = row['address']
    else:
        updated_address = row['address']

    # if pd.notna(updated_address) and 'kraków' not in updated_address.lower():
    #     updated_address += ", Kraków, małopolskie"
    
    return updated_address

sorted_filtered_df_otodom['address'] = sorted_filtered_df_otodom.apply(update_address, axis=1)
sorted_filtered_df_otodom['address'] = sorted_filtered_df_otodom['address'].str.replace(r'^ul\.\s+', '', regex=True)
sorted_filtered_df_otodom['address'] = sorted_filtered_df_otodom['address'].str.replace(r',\s*ul\.\s[^,\d]+,\s', ', ', regex=True)

sorted_filtered_df_otodom


,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,...,form of ownership,condition,rent,heating,advertiser type,elevator,outdoor area,building type,building material,is_duplicate
0,https://www.otodom.pl/pl/oferta/nowe-mieszkani...,Nowe mieszkania na Górce Narodowej,"29 listopada - okolice, Górka Narodowa, Prądni...",778689.0,61.30,12703.0,1,5,2,2023.0,...,pełna własność,NaN,NaN,miejskie,biuro nieruchomości,True,balkon,brak informacji,brak informacji,False
1,https://www.otodom.pl/pl/oferta/balkon-3-pokoj...,balkon + 3 pokoje + parking,"Adama Vetulaniego, Prądnik Biały, Prądnik Biał...",970000.0,55.00,17636.0,4,7,3,2023.0,...,pełna własność,NaN,450.0,miejskie,biuro nieruchomości,True,balkon,blok,brak informacji,False
3788,https://www.otodom.pl/pl/oferta/zlocien-2-poko...,"Złocień, 2 pokoje, balkon","Agatowa, Domagały,, Złocień, Bieżanów-Prokocim...",625000.0,36.80,16984.0,4,4,2,2019.0,...,pełna własność,do zamieszkania,425.0,miejskie,biuro nieruchomości,True,balkon,blok,brak informacji,True
2,https://www.otodom.pl/pl/oferta/komfortowe-2-p...,komfortowe 2-pokojowe mieszkanie 42m2,"Albertyńskie, Bieńczyce, Bieńczyce, Kraków, ma...",559000.0,42.00,13310.0,2,4,2,1964.0,...,NaN,NaN,500.0,miejskie,biuro nieruchomości,False,balkon,blok,brak informacji,False
3,https://www.otodom.pl/pl/oferta/aleja-29-listo...,Aleja 29 Listopada obok UR,"Aleja 29 Listopada, Górka Narodowa, Prądnik Bi...",695000.0,48.10,14449.0,3,3,2,NaN,...,pełna własność,do zamieszkania,NaN,gazowe,biuro nieruchomości,False,balkon,blok,brak informacji,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3779,https://www.otodom.pl/pl/oferta/2-pokojowe-mie...,"2-pokojowe mieszkanie 36,75 m2 ul.Siemaszki","Siemaszki, Żabiniec, Prądnik Biały, Kraków, ma...",735000.0,36.75,20000.0,0,4,2,1970.0,...,pełna własność,do zamieszkania,530.0,miejskie,prywatny,False,balkon,blok,brak informacji,False
3785,https://www.otodom.pl/pl/oferta/3-pokoje-na-za...,3 pokoje na Żabińcu !!!,"Żabiniec, Prądnik Biały, Kraków, małopolskie",850000.0,77.00,11039.0,4,6,3,2002.0,...,pełna własność,NaN,800.0,miejskie,biuro nieruchomości,True,balkon,blok,brak informacji,False
5716,https://www.otodom.pl/pl/oferta/mieszkanie-73-...,"Mieszkanie, 73 m², Kraków","Żabiniec, Prądnik Biały, Kraków, małopolskie",1100000.0,73.00,15068.0,0,6,3,2005.0,...,pełna własność,NaN,700.0,miejskie,biuro nieruchomości,True,balkon,blok,brak informacji,True
3786,https://www.otodom.pl/pl/oferta/widokowe-m3-z-...,"Widokowe M3 z klimatyzacją i parkingiem, Żabiniec","Żabiniec, Prądnik Biały, Kraków, małopolskie",1179000.0,71.50,16490.0,5,8,3,2001.0,...,pełna własność,do zamieszkania,940.0,miejskie,biuro nieruchomości,True,balkon,apartamentowiec,brak informacji,False


In [144]:
# Usuń adresy spoza Krakowa
sorted_filtered_df_otodom = sorted_filtered_df_otodom[sorted_filtered_df_otodom['address'].str.contains("Kraków", na=False)]

## Geolokacja

In [145]:
unique_addresses_otodom = set(sorted_filtered_df_otodom['address'].unique())
len(unique_addresses_otodom)

1605

In [146]:
from geopy.geocoders import Nominatim
import time

geolocator = Nominatim(user_agent="AGH_Student_Data_Exploration_Project")

def geocode(address):
    timeout = 10
    attempts = 0
    while attempts < 3:
        try:
            location = geolocator.geocode(address, timeout=timeout)
            if location:
                return pd.Series([location.address, location.latitude, location.longitude])
            else:
                return pd.Series([None, None, None])
        except Exception as e:
            print(f"Retry {attempts + 1}: Error during geocoding for address '{address}' - {str(e)}")
            attempts += 1
            timeout += 5
            time.sleep(2)
    return pd.Series([None, None, None])

In [147]:
geocode("Śląska, Kraków, małopolskie")

0    Śląska, Nowy Kleparz, Krowodrza, Kraków, wojew...
1                                            50.074171
2                                            19.933102
dtype: object

In [148]:
location_df = pd.DataFrame(unique_addresses_otodom, columns=['unique_addresses'])
small_df = location_df.head(10).copy()
small_df[['location', 'latitude', 'longitude']] = small_df['unique_addresses'].apply(geocode)
small_df

# location_df.to_csv('geocoded_addresses.csv', index=False)
# print("DataFrame saved to 'geocoded_addresses.csv'.")

,unique_addresses,location,latitude,longitude
0,"Na Ogrody, Prądnik Biały, Prądnik Biały, Krakó...","Ogrody, Bronowice Wielkie, Prądnik Biały, Krak...",50.098665,19.881480
1,"Józefa Dietla 15, Kazimierz, Stare Miasto, Kra...","15, Józefa Dietla, Kazimierz, Stare Miasto, Kr...",50.050373,19.938754
2,"Piastowska, Małe Błonia, Krowodrza, Kraków, ma...","Piastowska, Małe Błonia, Krowodrza, Kraków, wo...",50.070005,19.903898
3,"al. 29 Listopada, Prądnik Czerwony, Prądnik Cz...","Aleja 29 Listopada, Rogatka Warszawska, Olsza,...",50.082640,19.951981
4,"Krowoderska, Stare Miasto, Stare Miasto, Krakó...","Krowoderska, Piasek, Stare Miasto, Kraków, woj...",50.067927,19.937477
5,"Feliksa Konecznego, Żabiniec, Prądnik Biały, K...","Feliksa Konecznego, Osiedle Żabiniec, Prądnik ...",50.084295,19.941917
6,"Parkowa, Stare Podgórze, Podgórze, Kraków, mał...","Parkowa, Stare Podgórze, Podgórze, Kraków, woj...",50.042718,19.950724
7,"Półkole, Dąbie, Grzegórzki, Kraków, małopolskie","Półkole, Dąbie, Grzegórzki, Kraków, województw...",50.060536,19.981721
8,"Astronautów, Bielany, Zwierzyniec, Kraków, mał...","Astronautów, Osiedle Wojskowe, Bielany, Zwierz...",50.043882,19.820011
9,"29, Olsza, Prądnik Czerwony, Kraków, małopolskie","Olsza, Prądnik Czerwony, Kraków, województwo m...",50.082175,19.966456


# Zbiór danych nieruchomosci-online

### Załadowanie i agregacja danych

In [149]:
df_nieruchomosci = pd.read_csv('datasets/nieruchomosci-online_dataset_raw.csv')
df_2024_03_25_nieruchomosci_new_data = pd.read_csv('datasets/timestamps/2024-03-25-nieruchomosci-online-new_data.csv')
df_2024_03_25_nieruchomosci_updated_data = pd.read_csv('datasets/timestamps/2024-03-25-nieruchomosci-online-updated_data.csv')
df_2024_04_07_nieruchomosci_new_data = pd.read_csv('datasets/timestamps/2024-04-07-nieruchomosci-online-new_data.csv')
df_2024_04_07_nieruchomosci_updated_data = pd.read_csv('datasets/timestamps/2024-04-07-nieruchomosci-online-updated_data.csv')
df_2024_04_21_nieruchomosci_new_data = pd.read_csv('datasets/timestamps/2024-04-21-nieruchomosci-online-new_data.csv')
df_2024_04_21_nieruchomosci_updated_data = pd.read_csv('datasets/timestamps/2024-04-21-nieruchomosci-online-updated_data.csv')

In [150]:
df_nieruchomosci = update_dataframe(df_nieruchomosci, df_2024_03_25_nieruchomosci_updated_data, df_2024_03_25_nieruchomosci_new_data)
df_nieruchomosci = update_dataframe(df_nieruchomosci, df_2024_04_07_nieruchomosci_updated_data, df_2024_04_07_nieruchomosci_new_data)
df_nieruchomosci = update_dataframe(df_nieruchomosci, df_2024_04_21_nieruchomosci_updated_data, df_2024_04_21_nieruchomosci_new_data)

In [151]:
df_nieruchomosci.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8753 entries, 0 to 8752
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   url                                  8753 non-null   object 
 1   name/title                           8753 non-null   object 
 2   address                              8454 non-null   object 
 3   price                                8657 non-null   object 
 4   area                                 8753 non-null   object 
 5   price-per-area                       8657 non-null   object 
 6   floor/store                          8753 non-null   object 
 7   no of floors/stores in the building  8209 non-null   float64
 8   no of rooms                          8753 non-null   object 
 9   year of construction                 8753 non-null   object 
 10  parking space                        8753 non-null   object 
 11  market                        

### Cena i cena za metr
#### Usunięcie wierszy z brakującymi danymi

In [152]:
print(df_nieruchomosci[df_nieruchomosci['price'].isnull()].shape[0])
df_nieruchomosci.drop(df_nieruchomosci[df_nieruchomosci['price'].isnull()].index, inplace=True)

96


In [153]:
# sprawdzenie, czy są jakieś wartości w kolumnie price-per-area, które nie zawierają danych liczbowych
df_nieruchomosci[~df_nieruchomosci['price-per-area'].str.contains(r'\d')].shape[0]

0

In [154]:
# sprawdzenie, czy są jakieś wartości w kolumnie price, które nie zawierają danych liczbowych
df_nieruchomosci[~df_nieruchomosci['price'].str.contains(r'\d')].shape[0]

0

### Zamiana cen podanych w innych walutach

In [155]:
# wartości w kolumnie 'price' podane w innej walucie
# price-per-area w innej walucie

# df_nieruchomosci[~df_nieruchomosci['price'].str.contains('zł')].copy()
# df_nieruchomosci[~df_nieruchomosci['price-per-area'].str.contains('zł')].copy()

different_currency_price = pd.concat([df_nieruchomosci[~df_nieruchomosci['price'].str.contains('zł')].copy(), df_nieruchomosci[~df_nieruchomosci['price-per-area'].str.contains('zł')].copy()]).drop_duplicates()
different_currency_price

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership
153,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Piłsudskiego","Piłsudskiego, Stare Miasto, Kraków, małopolskie",1 149 999 €,"232,10 m²","4 954,76 €/m²",4,4.0,5,1906,parking strzeżony w pobliżu,wtórny,NaN
1120,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Piłsudskiego","Piłsudskiego, Stare Miasto, Kraków, małopolskie",1 149 999 €,"232,10 m²","4 954,76 €/m²",4,4.0,5,1908,garaż,wtórny,NaN
1151,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Piłsudskiego","Piłsudskiego, Stare Miasto, Kraków, małopolskie",1 150 000 €,"232,10 m²","4 954,76 €/m²",4,5.0,5,1906,parking publiczny / na ulicy,wtórny,NaN
2096,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Piłsudskiego","Piłsudskiego, Stare Miasto, Kraków, małopolskie",1 150 000 €,228 m²,"5 043,86 €/m²",4,5.0,5,1906,parking publiczny / na ulicy,wtórny,własność
2241,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Piłsudskiego","Piłsudskiego, Nowy Świat, Kraków, małopolskie",1 150 000 €,"228,70 m²","5 028,42 €/m²",4,5.0,5,1906,-,wtórny,NaN
2562,https://krakow.nieruchomosci-online.pl/mieszka...,Apartament Kraków,"Kraków, małopolskie",138 000 €,65 m²,"2 123,08 €/m²",21,45.0,2,2023,-,wtórny,NaN
2927,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, Plac Juliusza Kossaka","Plac Juliusza Kossaka, Stare Miasto, Kraków, m...",700 000 €,152 m²,"4 605,26 €/m²",1,4.0,4,1930,-,wtórny,NaN
2991,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"marszałka JózefaPiłsudskiego, Wawel, Kraków, m...",1 150 000 €,232 m²,"4 956,90 €/m²",4,5.0,9,1906,-,wtórny,NaN
3364,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie Stare Miasto, ul. Jana","Jana, Stare Miasto, Kraków, małopolskie",1 000 000 €,"221,19 m²",4 521 €/m²,4,4.0,4,1850,-,wtórny,NaN
8480,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Mazowiecka","Mazowiecka, Krowodrza, Kraków, małopolskie",370 000 €,84 m²,"4 404,76 €/m²",5,6.0,3,1936,-,wtórny,NaN


In [156]:
# zamiana cen na wartości liczbowe
df_nieruchomosci['price'] = df_nieruchomosci['price'].str.replace(' ', '').str.replace(',', '.').str.extract('(\d+\.\d+|\d+)').astype(float)

df_nieruchomosci['price-per-area'] = df_nieruchomosci['price-per-area'].str.replace(' ', '').str.replace(',', '.').str.extract('(\d+\.\d+|\d+)').astype(float)

In [157]:
print(df_nieruchomosci.loc[different_currency_price.index, ['price', 'price-per-area']])

          price  price-per-area
153   1149999.0         4954.76
1120  1149999.0         4954.76
1151  1150000.0         4954.76
2096  1150000.0         5043.86
2241  1150000.0         5028.42
2562   138000.0         2123.08
2927   700000.0         4605.26
2991  1150000.0         4956.90
3364  1000000.0         4521.00
8480   370000.0         4404.76


In [158]:
# kurs na 5.04.2024
# euro_exchange_rate = 4.28

df_nieruchomosci.loc[different_currency_price.index, 'price'] *= euro_exchange_rate
df_nieruchomosci.loc[different_currency_price.index, 'price-per-area'] *= euro_exchange_rate

In [159]:
df_nieruchomosci.loc[different_currency_price.index, ['price', 'price-per-area']]

,price,price-per-area
153,4.951232e+06,21332.337663
1120,4.951232e+06,21332.337663
1151,4.951236e+06,21332.337663
2096,4.951236e+06,21715.950853
2241,4.951236e+06,21649.475122
2562,5.941484e+05,9140.757463
2927,3.013796e+06,19827.592325
2991,4.951236e+06,21341.551269
3364,4.305423e+06,19464.817383
8480,1.593007e+06,18964.355013


### Usunięcie wierszy z przesunięciem danych w kolumnach

In [160]:

print(df_nieruchomosci['no of rooms'].str.contains('-', na=False).sum())
df_nieruchomosci.drop(df_nieruchomosci[df_nieruchomosci['no of rooms'].str.contains('-', na=False)].index, inplace=True)


18


In [161]:
print(df_nieruchomosci['no of rooms'].str.contains(r'\D', na=False).sum())
df_nieruchomosci.drop(df_nieruchomosci[df_nieruchomosci['no of rooms'].str.contains(r'\D', na=False)].index, inplace=True)


57


### Liczba pokoji, i pięter

#### Zamiana wartości w kolumnach na typ liczbowy

In [162]:
df_nieruchomosci['no of rooms'] = df_nieruchomosci['no of rooms'].astype(int)

In [163]:
df_nieruchomosci['area'] = df_nieruchomosci['area'].str.replace(' m²', '').str.replace(',', '.').astype(float)

In [164]:
df_nieruchomosci[df_nieruchomosci['floor/store'].str.contains('parter')]

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership
4,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Pustynna","Pustynna, Kliny, Kraków, małopolskie",1200000.0,71.06,16887.14,parter,1.0,3,2013,garaż w bryle budynku,wtórny,"własność, księga wieczysta"
5,https://krakow.nieruchomosci-online.pl/mieszka...,Apartament Kraków,"Wola Justowska, Kraków, małopolskie",1198680.0,71.35,16800.00,parter,2.0,4,2023,garaż w bryle budynku,pierwotny (zobacz inne nowe mieszkania w Krako...,własność
9,https://krakow.nieruchomosci-online.pl/mieszka...,"Apartament, ul. Szablowskiego","Szablowskiego, Bronowice, Kraków, małopolskie",599000.0,40.00,14975.00,parter,3.0,2,2001,tak,wtórny,"własność, księga wieczysta"
12,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Ułanów","Ułanów, Prądnik Czerwony, Kraków, małopolskie",790000.0,58.00,13620.69,parter,10.0,3,1962,-,wtórny,"własność, księga wieczysta"
18,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Obozowa","Obozowa, Ruczaj, Kraków, małopolskie",689900.0,51.94,13282.63,parter,7.0,2,2012,przynależne na ulicy,wtórny,"własność, księga wieczysta"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8712,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Szymańskiego","Szymańskiego, Branice, Kraków, małopolskie",371400.0,30.95,12000.00,parter,4.0,2,2025,tak,pierwotny (zobacz inne nowe mieszkania w Krako...,NaN
8726,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Glogera","Glogera, Prądnik Biały, Kraków, małopolskie",800000.0,47.21,16945.56,parter,3.0,3,2021,tak,wtórny,NaN
8730,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Kołłątaja","Kołłątaja, Kraków, małopolskie",1059000.0,60.57,17483.90,parter,4.0,3,1908,przynależne na ulicy,wtórny,własność
8731,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Sławka","Sławka, Dzielnica XI Podgórze Duchackie, Krakó...",409999.0,31.00,13225.77,parter,2.0,2,2024,garaż wolnostojący,pierwotny (zobacz inne nowe mieszkania w Krako...,własność


In [165]:
# Zamiana wartości "parter" na 0 w kolumnie "floor/store"
df_nieruchomosci['floor/store'] = df_nieruchomosci['floor/store'].replace('parter', '0')

In [166]:
# Zamiana wartości "suterena" na -1 w kolumnie "floor/store"
df_nieruchomosci['floor/store'] = df_nieruchomosci['floor/store'].replace('suterena', '-1')

In [167]:
# usunięcie wierszy, które w kolumnie 'floor/store' zawierają dane nieliczbowe
print(df_nieruchomosci[~df_nieruchomosci['floor/store'].str.contains(r'\d', na=False)].shape[0])
df_nieruchomosci.drop(df_nieruchomosci[~df_nieruchomosci['floor/store'].str.contains(r'\d')].index, inplace=True)

94


In [168]:
# przesunięte dane
print(df_nieruchomosci[df_nieruchomosci['floor/store'].str.contains(r'[a-zA-Z]')].shape[0])
df_nieruchomosci.drop(df_nieruchomosci[df_nieruchomosci['floor/store'].str.contains(r'[a-zA-Z]')].index, inplace=True)

117


In [169]:
df_nieruchomosci['floor/store'] = df_nieruchomosci['floor/store'].astype(int)

In [170]:
print(df_nieruchomosci[df_nieruchomosci['no of floors/stores in the building'].isna()].shape[0])
df_nieruchomosci.dropna(subset=['no of floors/stores in the building'], inplace=True)
df_nieruchomosci['no of floors/stores in the building'] = df_nieruchomosci['no of floors/stores in the building'].astype(int)

177


In [184]:
# Według wikipedii najwyższy mieszkalny budunek w Krk ma 19 pięter
# https://pl.wikipedia.org/wiki/Lista_najwyższych_budynków_w_Krakowie
print(df_nieruchomosci[df_nieruchomosci['no of floors/stores in the building'] > 19].shape[0])
df_nieruchomosci.drop(df_nieruchomosci[df_nieruchomosci['no of floors/stores in the building'] > 19].index, inplace=True)

1


### Rok budowy

In [171]:
df_nieruchomosci['year of construction'] = pd.to_numeric(df_nieruchomosci['year of construction'], errors='coerce')

missing_below_1200 = df_nieruchomosci[df_nieruchomosci['year of construction'] < 1250]
missing_above_2026 = df_nieruchomosci[df_nieruchomosci['year of construction'] > 2026]
pd.concat([missing_above_2026, missing_below_1200])

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership
7393,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Banacha","Banacha, Prądnik Biały, Kraków, małopolskie",759000.0,49.47,15342.63,6,8,3,2924.0,tak,pierwotny (zobacz inne nowe mieszkania w Krako...,własność
2100,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Magnolii","Magnolii, Ruczaj, Kraków, małopolskie",790300.0,56.45,14000.00,3,3,3,1024.0,garaż,pierwotny (zobacz inne nowe mieszkania w Krako...,NaN


In [172]:
# Poprawianie "z placa"
df_nieruchomosci['year of construction'] = df_nieruchomosci['year of construction'].replace(2924.0, 2024)
df_nieruchomosci['year of construction'] = df_nieruchomosci['year of construction'].replace(1024.0, 2024)

In [173]:
df_nieruchomosci['year of construction'].value_counts(dropna=False)

year of construction
2024.0    1417
2023.0    1390
NaN        717
2025.0     660
2022.0     383
          ... 
1921.0       1
1946.0       1
1870.0       1
1350.0       1
1845.0       1
Name: count, Length: 153, dtype: int64

### Parking

In [174]:
df_nieruchomosci['parking space'] = df_nieruchomosci['parking space'].replace('-', 'parking publiczny / na ulicy')

In [175]:
df_nieruchomosci['parking space'].unique()

array(['tak', 'w garażu podziemnym', 'parking publiczny / na ulicy',
       'garaż w bryle budynku', 'garaż', 'przynależne na ulicy',
       'przynależne na terenie ogrodzonym', 'możliwość wykupienia',
       'garaż wolnostojący', 'parking strzeżony w pobliżu',
       'wiata garażowa'], dtype=object)

In [176]:
df_nieruchomosci['parking space'].value_counts(dropna=False)

parking space
parking publiczny / na ulicy         3152
w garażu podziemnym                  1465
tak                                  1044
garaż                                 794
przynależne na ulicy                  582
garaż wolnostojący                    417
przynależne na terenie ogrodzonym     224
garaż w bryle budynku                 205
parking strzeżony w pobliżu           173
możliwość wykupienia                  113
wiata garażowa                         25
Name: count, dtype: int64

In [177]:
# Skopiowanie danych do nowej kolumny (mogą się przydać do jakiegoś modelu) i konwersja na boolean dla spójności z otodom
df_nieruchomosci['parking space details'] = df_nieruchomosci['parking space']
false_values = ['parking publiczny / na ulicy', 'możliwość wykupienia', 'parking strzeżony w pobliżu']
df_nieruchomosci['parking space'] = ~df_nieruchomosci['parking space'].isin(false_values)

### Rynek

In [178]:
valid_values = ['wtórny', 'pierwotny (zobacz inne nowe mieszkania w Krakowie)']

shifted_rows = ~df_nieruchomosci['market'].isin(valid_values) & df_nieruchomosci['market'].notna()

df_nieruchomosci.drop(df_nieruchomosci[shifted_rows].index, inplace=True)

In [179]:
df_nieruchomosci.loc[(df_nieruchomosci['market'].isna()) & (df_nieruchomosci['year of construction'] > 2022), 'market'] = 'pierwotny'
df_nieruchomosci.loc[(df_nieruchomosci['market'].isna()) & (df_nieruchomosci['year of construction'] < 2023), 'market'] = 'wtórny'
print(df_nieruchomosci['market'].isna().sum())
df_nieruchomosci.dropna(subset=['market'], inplace=True)

33


### Forma własności

In [180]:
# Zgodnie z obowiązującym prawem 2007 r. nie ma możliwości ustanawiania spółdzielczego własnościowego prawa do lokalu
df_nieruchomosci.loc[(df_nieruchomosci['form of ownership'].isna()) & (df_nieruchomosci['year of construction'] > 2006), 'form of ownership'] = 'własność'

In [181]:
ownership_mapping = {
    'własność': 'pełna własność',
    'własność, księga wieczysta': 'pełna własność',
    'spółdzielcze własnościowe, księga wieczysta': 'spółdzielcze wł. prawo do lokalu',
    'spółdzielcze własnościowe': 'spółdzielcze wł. prawo do lokalu',
    'udział w KW': 'udział',
    'udział ze wskazaniem, KW na budynku': 'udział',
    'księga wieczysta': 'pełna własność',
    'inna': 'użytkowanie wieczyste / dzierżawa',
    'udział': 'udział',
    'inna, księga wieczysta': 'użytkowanie wieczyste / dzierżawa',
    'Udział ze wskazaniem': 'udział',
    'Udział': 'udział',
    'udział, księga wieczysta': 'udział',
    'udział ze wskazaniem': 'udział',
    'Chce dopłacić': np.nan,
    'tak': np.nan,
    'udziały, księga wieczysta': 'udział',
    'Oczekuje dopłaty': np.nan,
    'KW budynek, udział ze wskazaniem': 'udział',
    'Spółdzielcza wł. z KW': 'spółdzielcze wł. prawo do lokalu'
}

df_nieruchomosci['form of ownership'] = df_nieruchomosci['form of ownership'].replace(ownership_mapping)

# df_nieruchomosci['form of ownership'].fillna('NaN', inplace=True)


In [183]:
df_nieruchomosci['form of ownership'].value_counts(dropna=False)

form of ownership
pełna własność                       6404
NaN                                  1506
spółdzielcze wł. prawo do lokalu      200
użytkowanie wieczyste / dzierżawa      31
udział                                 12
Name: count, dtype: int64

In [187]:
nan_row_count = df_nieruchomosci.isna().any(axis=1).sum()
df_nieruchomosci.shape[0] - nan_row_count

6489

In [185]:
df_nieruchomosci.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8152 entries, 0 to 8752
Data columns (total 14 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   url                                  8152 non-null   object 
 1   name/title                           8152 non-null   object 
 2   address                              8152 non-null   object 
 3   price                                8152 non-null   float64
 4   area                                 8152 non-null   float64
 5   price-per-area                       8152 non-null   float64
 6   floor/store                          8152 non-null   int32  
 7   no of floors/stores in the building  8152 non-null   int32  
 8   no of rooms                          8152 non-null   int32  
 9   year of construction                 7469 non-null   float64
 10  parking space                        8152 non-null   bool   
 11  market                             

In [45]:
df_nieruchomosci.describe()

,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms
count,8.194000e+03,8194.000000,8194.000000,8194.000000,8194.000000,8194.000000
mean,9.840526e+05,59.009789,16838.296094,2.393337,4.867342,2.653649
std,6.568436e+05,31.140413,4702.280025,2.191137,2.479374,1.245671
min,1.490000e+05,11.700000,19.140000,-1.000000,1.000000,1.000000
25%,6.659775e+05,40.952500,13995.032500,1.000000,3.000000,2.000000
50%,8.191800e+05,52.030000,16155.000000,2.000000,4.000000,3.000000
75%,1.079973e+06,67.090000,18840.910000,4.000000,6.000000,3.000000
max,1.600000e+07,627.000000,71018.210000,21.000000,45.000000,44.000000


In [111]:
df_nieruchomosci.isnull().sum()

url                                       0
name/title                                0
address                                  59
price                                     0
area                                      0
price-per-area                            0
floor/store                               0
no of floors/stores in the building     158
no of rooms                               0
year of construction                      0
parking space                             0
market                                   28
form of ownership                      2450
dtype: int64